# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## 1-What test is appropriate for this problem? Does CLT apply?


In [5]:
w = data[data.race=='w']['call'].values
b = data[data.race=='b']['call'].values

n_w=len(w)
n_b=len(b)

p_w=np.sum(w)/n_w
p_b=np.sum(b)/n_b

the 2-sample z-test for proportions is appropriate for this problem.

In [6]:
#Check if CLT applies

#randomness condition is assumed.

#We check for the normal condition:

assert p_w*n_w>=10
assert p_b*n_b>=10
assert (1-p_w)*n_w>=10
assert (1-p_b)*n_b>=10

print('The sampling is assumed to be random, shown to satisfy the normal conditions, and assumed to '+
     'be much smaller than 10% of the total population')


The sampling is assumed to be random, shown to satisfy the normal conditions, and assumed to be much smaller than 10% of the total population


## 2-What are the null and alternate hypotheses?
The null hypothesis is that both b and w have the sample p (the proportions for both are the same, or the 
probability of success i.e. getting a callback is the same for both).

The alternate hypothesis is that the proportion for w is higher than the proportion for b

we set a significance level of 5%

Formally:

H0: p_w-p_b=0

Ha: p_w>p_b

alpha=0.05



In [7]:
# Your solution to Q3 here

## 3-Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches

We begin with the frequentist method:

Z=(p_w-p_b)/sqrt(p*(1-p)*((1/n_w)+(1/n_b)))

CI: (p_w-p_b) +/- Z**sqrt( p_w*q_w/n_w   +p_b*q_b/n_b )

where q_b=1-p_b and similarly for w




In [12]:
#b and w population sample proportion
p=(np.sum(b)+np.sum(w))/(n_w+n_b)

#z-statistic

z=(p_w-p_b)/np.sqrt(p*(1-p)*((1/n_w)+(1/n_b)))

#for 95% confidence interval we use z=1.96

CI=[p_w-p_b-1.96*np.sqrt((p_w*(1-p_w)/n_w)+(p_b*(1-p_b)/n_b)),p_w-p_b+1.96*np.sqrt((p_w*(1-p_w)/n_w)+(p_b*(1-p_b)/n_b))]

#calculate p-value
p_value=1-stats.norm.cdf(z)

print('CI: we are 95% confident that the population proprotion difference pw-pb is in: '+str(CI))
print('z statistic is: '+str(z))
print('p_value is: '+str(p_value))

CI: we are 95% confident that the population proprotion difference pw-pb is in: [0.016777447859559147, 0.047288260559332024]
z statistic is: 4.108412152434346
p_value is: 1.991943418788722e-05


We can thus reject the null hypothesis.

Bootstrap Method:

In [14]:
ObservedDiff=p_w-p_b

bw=np.concatenate((b,w))

N_Bootstraps=10000
bs_replicates=np.empty(N_Bootstraps)

for i in range(N_Bootstraps):
    b_bs=np.random.choice(bw,size=n_b)
    w_bs=np.random.choice(bw,size=n_w)
    bs_replicates[i]=(np.sum(w_bs)/n_w)-(np.sum(b_bs)/n_b)
    
CI=np.percentile(bs_replicates,[2.5,97.5])
p_value=np.sum(bs_replicates>=ObservedDiff)/len(bs_replicates)

print('confidence interval for difference assuming black and white have same proprotions: '+str(CI))
print('Observed proportion diff: ' +str(ObservedDiff))
print('p_value: '+str(p_value))
print('The null hypothesis is rejected')

#If we are interested in CI for true proprotions difference we can do the below:

bs_replicates2=np.empty(N_Bootstraps)

for i in range(N_Bootstraps):
    b_bs=np.random.choice(b,size=len(b))
    w_bs=np.random.choice(w,size=len(w))
    bs_replicates2[i]=(np.sum(w_bs)/n_w)-(np.sum(b_bs)/n_b)

CI=np.percentile(bs_replicates2,[2.5,97.5])
print('The 95% confidence interval for the true population proprotions diff is: '+str(CI))    

confidence interval for difference assuming black and white have same proprotions: [-0.01519507  0.01519507]
Observed proportion diff: 0.032032854209445585
p_value: 0.0001
The null hypothesis is rejected
The 95% confidence interval for the true population proprotions diff is: [0.01683778 0.04722793]




<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4.Write a story describing the statistical significance in the context or the original problem.

The above analysis indicates that with 95% confidence, we can conclude that the difference:

p_w-p_b is in the interval [0.016,0.047]

This means that we are confident with 95% certainty that the proprotion of whites getting call-backs is higher than that for blacks.



## 5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?


It does not. This analysis simply compares proprotions for blacks vs non-blacks. It does not assess the magnitude of the proprotion for call back given the applicant is white or black. Lets say we had another variable (say age) and the variable took values Old and Young. A possibls experiment would be to calculate the probabilities P(Callback|Young), P(Callback|Old), P(Callback|Black), P(Callback|White) (we could get a confidence interval for each of these values), and compare them. 

It would definitely be important to also see relations between these variables (i.e. what if all black applicants were old, and all white applicants were young).


